In [257]:
# Import packages
import pandas as pd
import numpy as np
import patsy
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.linear_model import lars_path
from sklearn.linear_model import LinearRegression, Lasso, LassoCV
from sklearn.metrics import r2_score
import scipy.stats as stats
# Visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
df=pd.read_csv("../data/airbnb.csv")
model_df=df

In [ ]:
model_df.corr() # Whole correlation matrix
model_df.corr()['log_price'] # Check correlations with outcome only
sns.heatmap(model_df.corr(), cmap="seismic", annot=True, vmin=-1, vmax=1);

In [ ]:
# Example dummy coding for 'cancellation_policy'
model_df = pd.get_dummies(model_df, columns=['cancellation_policy'], drop_first=True)

In [ ]:
sns.distplot(model_df['log_price'], kde=True,);
fig = plt.figure()
res = stats.probplot(model_df['log_price'], plot=plt)
print("Skewness: %f" % model_df['log_price'].skew())
print("Kurtosis: %f" % model_df['log_price'].kurt())

In [ ]:
# Create feature matrix with patsy- this way we get an intercept
y_census, X_census = patsy.dmatrices('log_price ~ accommodates + bathrooms + cleaning_fee +
                                     host_identity_verified + host_response_rate + instant_bookable + number_of_reviews + 
                                     review_scores_rating + bedrooms + beds + amenity_count + tourism_mentions + 
                                     cancellation_policy_moderate + cancellation_policy_strict + property_type_House + 
                                     property_type_Other + bed_type + room_type_Private_room + room_type_Shared_room + median_home_val + 
                                     median_income', data=model_df_with_census, return_type="dataframe")

#test_size 80-20 split
X_train_census, X_test_census, y_train_census, y_test_census = train_test_split(X_census, y_census, test_size=0.2,
random_state=42)
X_train_census.shape[0] + X_test_census.shape[0] == model_df_with_census.shape[0]

# Create model
model_census_sm = sm.OLS(y_train_census,X_train_census)

# Fit model to training set
fit_census = model_census_sm.fit()

# Print fit summary
fit_census.summary()

In [ ]:
# Check for VIFs of each feature, then save to its own DF
vif_census = pd.DataFrame()
vif_census[“VIF Factor”] = [variance_inflation_factor(X_census.values, i) for i in range(X.shape[1])]
vif_census[“features”] = X_census.columns

In [ ]:
# Use statsmodels to plot the residuals vs the fitted values
plt.figure(figsize=(12,8))
plt.scatter(fit_census.predict(), fit_census.resid); # print resids vs predictions
plt.title("Residuals plot from OLS Model")
plt.xlabel("Predicted Values")
plt.ylabel("Residuals")
plt.savefig('LR_Residual_Plot')

In [ ]:
# Standard scale the data
std = StandardScaler()
std.fit(X_train_census.values) # only std.fit on train set
X_tr_census = std.transform(X_train_census.values)
X_te_census = std.transform(X_test_census.values)

In [ ]:
# Run the cross validation, find the best alpha, refit the model on all the data with that alpha
alphavec = 10**np.linspace(-2,2,200)
lasso_model_census = LassoCV(alphas = alphavec, cv=5)
lasso_model_census.fit(X_tr_census, y_train_census)

In [ ]:
# Print feature name zipped with its beta
lasso_betas = list(zip(X_train_census.columns,
lasso_model_census.coef_))
# R2 of Training set
lasso_model_census.score(X_tr_census,y_train_census)

In [ ]:
# Predict model on test data
y_census_pred = lasso_model_census.predict(X_te_census)
# R2 of test set using this model
r2_score(y_test_census, y_census_pred)
#Mean Absolute Error (MAE)
def mae(y_true, y_pred):
    return np.mean(np.abs(y_pred - y_true))
mae(y_test_census, y_census_pred)
# Plot
plt.scatter(y_test_census, y_census_pred)
plt.plot([0,10],[0,10],color='red')
plt.grid(True)
plt.title('Predicted vs. Actual Rental Price (log) with LASSO CV')
plt.ylabel('Rental Price (log) Predicted')
plt.xlabel('Rental Price (log) Actual');

In [ ]:
print("Computing regularization path using the LARS ...")
alphas, _, coefs = lars_path(X_tr_census, y_train_census, method='lasso')
# # plotting the LARS path
xx = np.sum(np.abs(coefs.T), axis=1)
xx /= xx[-1]
plt.figure(figsize=(10,10))
plt.plot(xx, coefs.T)
ymin, ymax = plt.ylim()
plt.vlines(xx, ymin, ymax, linestyle='dashed')
plt.xlabel('|coef| / max|coef|')
plt.ylabel('Coefficients')
plt.title('LASSO Path')
plt.axis('tight')
plt.legend(X_train_census.columns)
plt.show()